In [1]:
!pip install /content/CloudPredictionFramework
!pip install -r /content/CloudPredictionFramework/requirements.txt

Processing ./CloudPredictionFramework
  Preparing metadata (setup.py) ... done
  Created wheel for cloudpredictionframework: filename=cloudpredictionframework-0.0.54-py3-none-any.whl size=16504 sha256=43f50d46ac88abc390012939e3e226c064e559aca1bb95bcb2b2722ef761efa9
  Stored in directory: /tmp/pip-ephem-wheel-cache-sr8__qnz/wheels/c0/f6/c3/f731d4e84f62dac3888b7eb6d4d2cd4b5ccbdc22f7d3ea82bf
Successfully built cloudpredictionframework
  Attempting uninstall: cloudpredictionframework
    Found existing installation: cloudpredictionframework 0.0.54
    Uninstalling cloudpredictionframework-0.0.54:
      Successfully uninstalled cloudpredictionframework-0.0.54


In [2]:
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale
import pandas as pd
import traceback
from os import listdir, mkdir
from os.path import isfile, join, isdir, dirname, abspath

In [7]:
import os
print(os.getcwd())
!git clone https://github.com/filga/CloudPredictionFramework.git

/content
Cloning into 'CloudPredictionFramework'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 167 (delta 32), reused 117 (delta 28), pack-reused 30 (from 1)
Receiving objects: 100% (167/167), 165.18 KiB | 4.13 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [3]:
# Get the root directory
root_dir = dirname(abspath('.'))

# Specify the base path
BASE_PATH = join(root_dir, '/content/CloudPredictionFramework/examples/dataset')

isdir(BASE_PATH)

True

In [ ]:
#@title Data Provider Setup
class ExperimentDataProvider:
    """
    This class is used to provide data for the experiments.
    """

    # Get the root directory
    root_dir = dirname(abspath('.'))

    # Specify the base path
    BASE_PATH = join(root_dir, '/content/CloudPredictionFramework/examples/dataset')

    def __init__(self):
        if not isdir(self.BASE_PATH):
            raise FileNotFoundError('please create folder vm_data with vm csvs')

        self.csv_names = [f for f in listdir(self.BASE_PATH) if isfile(join(self.BASE_PATH, f))]
        self._experiments_data_count = {'DEMO': len(self.csv_names)}

    def get_experiment_count(self):
        return len(self._experiments_data_count)

    def get_experiment_len(self, experiment):
        return self._experiments_data_count[experiment]

    def get_experiment_names(self):
        return self._experiments_data_count.keys()

    def get_experiment_generator(self, experiment, metric):
        if experiment not in self._experiments_data_count:
            return None
        else:
            return self._experiment_gen(experiment, self._experiments_data_count[experiment], metric)

    def _experiment_gen(self, experiment, exp_num, metric):
        for i in range(exp_num):
            try:
                df = pd.read_csv(join(self.BASE_PATH, self.csv_names[i]), parse_dates=['timestamp'])
                if metric not in df:
                    continue
                df.dropna(inplace=True)
                df.reset_index(inplace=True)
                df[metric] = minmax_scale(df[metric], feature_range=(0, 100))
                yield df
            except Exception as e:
                print(traceback.format_exc())

In [27]:
dp.get_experiment_generator('DEMO', 'cpu.usage.average')

<generator object ExperimentDataProvider._experiment_gen at 0x7dcabe4c4ac0>

In [33]:
#@title Launch the experiments
from cloudpredictionframework.utils.experiment import run_batch_experiment
from cloudpredictionframework.utils.analysis import analyze_batch_result
from cloudpredictionframework.anomaly_detection.algorithms import LimitAlgorithm, MedianAlgorithm, KalmanAlgorithm, SavgolAlgorithm, WeightedHybridAlgorithm
import json
import numpy as np

RESULTS_PATH = 'experiment_results'
if not isdir(RESULTS_PATH):
    mkdir(RESULTS_PATH)

dp = ExperimentDataProvider()


completed_experiments = []

for exp_name in dp.get_experiment_names():
    dfs = ExperimentDataProvider().get_experiment_generator(exp_name, 'cpu.usage.average')

    print('Experiment', exp_name, 'Length:', dp.get_experiment_len(exp_name))
    hybrid = WeightedHybridAlgorithm([(MedianAlgorithm(), 0.9),
                                    (LimitAlgorithm(upper_treshold=20, lower_treshold=0), 0.3),
                                    (KalmanAlgorithm(tolerance_multiplier=1), 0.5),
                                    (SavgolAlgorithm(), 0.7)])


    resb = run_batch_experiment(dfs, [LimitAlgorithm(upper_treshold=50, lower_treshold=0),
                                    MedianAlgorithm(),
                                    hybrid])

    try:
        with open(join(OUTPUT_PATH, exp_name + '.json'), 'w') as fp:
            json.dump(resb, fp)
        with open(join(OUTPUT_PATH, exp_name + '.json'), 'w') as fp:
            json.dump(resb, fp)
        print('Successfully completed experiment', exp_name)
    except Exception:
        print(traceback.format_exc())
        print('Failed to save data for experiment', exp_name)

Experiment DEMO Length: 99


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
1it [00:08,  8.74s/it]

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/cloudpredictionframework/utils/experiment.py", line 98, in run_batch_experiment
    result = run_experiment(df, LSTM2Layer(input_shape=(1, 4)), **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/cloudpredictionframework/utils/experiment.py", line 34, in run_experiment
    y_pred = sc.inverse_transform(network.predict(test))
  File "/usr/local/lib/python3.10/dist-packages/cloudpredictionframework/usage_prediction/networks/base_network.py", line 16, in predict
    return self.model.predict(data)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py", line 244, in _adjust_input_rank
    raise ValueError(
ValueError: Exception encountered when calling Sequential.call().

Invalid input shape for input Tensor("data:0", 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2it [00:16,  8.22s/it]

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/cloudpredictionframework/utils/experiment.py", line 98, in run_batch_experiment
    result = run_experiment(df, LSTM2Layer(input_shape=(1, 4)), **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/cloudpredictionframework/utils/experiment.py", line 34, in run_experiment
    y_pred = sc.inverse_transform(network.predict(test))
  File "/usr/local/lib/python3.10/dist-packages/cloudpredictionframework/usage_prediction/networks/base_network.py", line 16, in predict
    return self.model.predict(data)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py", line 244, in _adjust_input_rank
    raise ValueError(
ValueError: Exception encountered when calling Sequential.call().

Invalid input shape for input Tensor("data:0", 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2it [00:19,  9.64s/it]


KeyboardInterrupt: 

In [16]:
type(dp)

__main__.ExperimentDataProvider

In [ ]:
#@title (Optional) Reload the experiments results
import json
from os.path import isfile, join

RESULTS_PATH = 'experiment_results' #@param {type: "string"}
with open(join(RESULTS_PATH, 'DEMO.json'), 'r') as fp:
    resb = json.load(fp)

In [ ]:
#@title heatmap and results analysis
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import json
from cloudpredictionfamework.usage_prediction.utils import analyze_batch_result


resba = analyze_batch_result(resb)

frames = []
for key in resba.keys():
    frames.append(pd.DataFrame(resba[key], index=[key]))
resdf = pd.concat(frames)

resdf2 = StandardScaler().fit_transform(resdf)

# fig1 = px.imshow(resdf,
#                 x=list(resdf.columns),
#                 y=list(resdf.index),
#                 color_continuous_scale='RdBu_r')
# fig1.show()

fig2 = px.imshow(resdf2,
                x=list(resdf.columns),
                y=list(resdf.index),
                color_continuous_scale='RdBu_r')
fig2.show()

asd = pd.DataFrame.from_dict(resba, orient='index')
asd